In [73]:
# %load_ext iminizinc
%load_ext autoreload
%autoreload 2

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
# import ipywidgets as widgets
from ipywidgets import *
from IPython.display import display
import IPython
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np

from datetime import datetime
# import talib
import pandas_ta as ta
# from talib.abstract import *
from math import *
from collections import OrderedDict
# import vectorbt as vbt    
import json
from decimal import *
getcontext().prec = 6
import seaborn as sns
from scipy.signal import find_peaks, peak_widths
import pickle
import re


import sys, os
sys.path.append(os.path.abspath('..'))
from research.pkg.lib import *


# import mplfinance as mpf
%matplotlib inline

# plt.rcParams['lines.linewidth'] = 0.5 



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [96]:

# Machine learning libraries
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from sklearn.linear_model import Ridge, Lasso, LinearRegression


from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [5]:
pair = 'BTC_USDT'
timeframe = '4h'

exchange = 'binance'
odf = pd.read_json(f'../freq-user-data/data/{exchange}/{pair}-{timeframe}.json'
# exchange = 'kucoin'
# odf = pd.read_json(f'../../freq-user-data/data/{exchange}/futures/{pair}-{timeframe}-futures.json'
).dropna().set_axis(['timestamp', 'open', 'high', 'low', 'close', 'volume'], axis=1
).assign(dtime=lambda x: pd.to_datetime(x['timestamp'], unit='ms', utc=False)
).set_index('dtime').sort_index()

print(odf.shape)
odf.head(4)

(12676, 6)


,timestamp,open,high,low,close,volume
dtime,,,,,,
2017-08-17 04:00:00,1502942400000,4261.48,4349.99,4261.32,4349.99,82.088865
2017-08-17 08:00:00,1502956800000,4333.32,4485.39,4333.32,4427.30,63.619882
2017-08-17 12:00:00,1502971200000,4436.06,4485.39,4333.42,4352.34,174.562001
2017-08-17 16:00:00,1502985600000,4352.33,4354.84,4200.74,4325.23,225.109716


In [160]:
### Load it all. Period.

idf = odf.copy().ffill()

# body low and highs
idf['body_high'] = idf[['open', 'close']].max(axis=1)
idf['body_low'] = idf[['open', 'close']].min(axis=1)

# log return
idf['lret'] = idf.close.divide(idf.close.shift()).apply(np.log)
# candle price direction, up or down (1,-1)
idf['candle_dir'] = 1 - 2*(idf.lret < 0)
# wicks log percent form candle
idf['wiLdist'] = idf.low.divide(idf.body_low).apply(np.log)
idf['wiHdist'] = idf.high.divide(idf.body_high).apply(np.log)
# number of down candles in last half, 1, 3 and 7 days
for days in [0.5, 1, 3, 7]:
    idf[f'n_down_{str(days)}'] = (idf.candle_dir < 0).astype(int).rolling(int(days*6)).sum().abs()
# distance from peaks in last periods
# dist_lags = [d*4 for d in [1,3,7,14,30]]
dist_lags = [1,3,7,14,30]
for dlag in dist_lags:
    idf[f'Hladist_{dlag}'] = idf.close.divide(idf.close.rolling(dlag).max()).apply(np.log)
    idf[f'Lladist_{dlag}'] = idf.close.divide(idf.close.rolling(dlag).min()).apply(np.log)
# rolling standard deviations
std_lags = [d*4 for d in [1,2,3,7,14,30]]
for la in std_lags:
    idf[f'std_{la}'] = idf.close.rolling(la).std()
    idf[f'distEma_{la}'] = idf.close.divide(idf.close.ewm(span=la).mean()).apply(np.log)
    idf[f'val_{la}'] = idf[f'distEma_{la}'] / idf[f'std_{la}']
# distance from emas
# ema_lags = [5, 9, 14, 21, 30]
# for ema in ema_lags:
#     idf[f'distEma_{ema}'] = idf.close.divide(idf.close.ewm(span=ema).mean()).apply(np.log)

# future returns for close and max/min high/low
future_periods = [1,2,3,6,12,18,42]
for fu in future_periods:
    idf[f'flretC_{fu}'] = idf.close.shift(-fu).divide(idf.close).apply(np.log)
    idf[f'flretH_{fu}'] = idf.high.rolling(fu).max().shift(-fu).divide(idf.close).apply(np.log)
    idf[f'flretL_{fu}'] = idf.high.rolling(fu).max().shift(-fu).divide(idf.close).apply(np.log)

idf.to_csv('test.csv')
idf = idf.dropna()
idf

,timestamp,open,high,low,close,volume,body_high,body_low,lret,candle_dir,...,flretL_6,flretC_12,flretH_12,flretL_12,flretC_18,flretH_18,flretL_18,flretC_42,flretH_42,flretL_42
dtime,,,,,,,,,,,,,,,,,,,,,
2017-09-06 00:00:00,1504656000000,4366.49,4522.81,4365.73,4430.02,187.420569,4430.02,4366.49,0.014449,1,...,0.051227,0.038749,0.077832,0.077832,-0.043367,0.077832,0.077832,-0.118026,0.077832,0.077832
2017-09-06 04:00:00,1504670400000,4448.10,4505.96,4335.26,4461.29,166.307942,4461.29,4448.10,0.007034,1,...,0.044193,0.034742,0.070798,0.070798,-0.031831,0.070798,0.070798,-0.125842,0.070798,0.070798
2017-09-06 08:00:00,1504684800000,4476.13,4597.79,4469.82,4548.13,175.294820,4548.13,4476.13,0.019278,1,...,0.024915,0.021620,0.051520,0.051520,-0.049696,0.051520,0.051520,-0.171329,0.051520,0.051520
2017-09-06 12:00:00,1504699200000,4544.59,4619.47,4422.28,4619.43,261.596797,4619.43,4544.59,0.015555,1,...,0.029819,-0.060406,0.035965,0.035965,-0.077253,0.035965,0.035965,-0.196587,0.035965,0.035965
2017-09-06 16:00:00,1504713600000,4619.43,4619.43,4619.43,4619.43,0.000000,4619.43,4619.43,0.000000,1,...,0.035965,-0.097568,0.035965,0.035965,-0.082482,0.035965,0.035965,-0.174485,0.035965,0.035965
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-25 16:00:00,1685030400000,26226.12,26631.98,26207.47,26468.68,7740.723170,26468.68,26226.12,0.009206,1,...,0.017359,0.009809,0.017359,0.017359,0.039169,0.047141,0.047141,0.014776,0.072085,0.072085
2023-05-25 20:00:00,1685044800000,26468.67,26551.02,26432.46,26473.79,2892.628090,26473.79,26468.67,0.000193,1,...,0.017166,0.014270,0.017166,0.017166,0.058368,0.065339,0.065339,0.012916,0.071892,0.071892
2023-05-26 00:00:00,1685059200000,26473.80,26539.34,26350.00,26435.14,4447.142980,26473.80,26435.14,-0.001461,-1,...,0.018627,0.028053,0.030253,0.030253,0.055902,0.073353,0.073353,0.021143,0.073353,0.073353


In [161]:
# from summarytools import dfSummary

In [162]:
# idf.wiLdist.quantile(np.logspace(-3,log(1),11))
# list(np.linspace(0,1,5))
# idf.wiLdist.quantile(np.linspace(0,1,11))

In [163]:
# idf.iloc[:,8:36].columns
features = [c for c in idf.columns if c not in ['timestamp','open','high','low','close','body_high','body_low',] and not re.match(r'^flret', c)]

features

# idf.columns

['volume',
 'lret',
 'candle_dir',
 'wiLdist',
 'wiHdist',
 'n_down_0.5',
 'n_down_1',
 'n_down_3',
 'n_down_7',
 'Hladist_1',
 'Lladist_1',
 'Hladist_3',
 'Lladist_3',
 'Hladist_7',
 'Lladist_7',
 'Hladist_14',
 'Lladist_14',
 'Hladist_30',
 'Lladist_30',
 'std_4',
 'distEma_4',
 'val_4',
 'std_8',
 'distEma_8',
 'val_8',
 'std_12',
 'distEma_12',
 'val_12',
 'std_28',
 'distEma_28',
 'val_28',
 'std_56',
 'distEma_56',
 'val_56',
 'std_120',
 'distEma_120',
 'val_120']

In [125]:
X = idf[features]
y = idf.flretH_3
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   random_state = 0)


In [131]:
features

['volume',
 'lret',
 'candle_dir',
 'wick_low_lret',
 'wick_high_lret',
 'n_down_0.5',
 'n_down_1',
 'n_down_3',
 'n_down_7',
 'Hladist_4',
 'Lladist_4',
 'Hladist_12',
 'Lladist_12',
 'Hladist_28',
 'Lladist_28',
 'Hladist_56',
 'Lladist_56',
 'Hladist_120',
 'Lladist_120',
 'std_4',
 'distEma_4',
 'val_4',
 'std_8',
 'distEma_8',
 'val_8',
 'std_12',
 'distEma_12',
 'val_12',
 'std_28',
 'distEma_28',
 'val_28',
 'std_56',
 'distEma_56',
 'val_56',
 'std_120',
 'distEma_120',
 'val_120']

In [165]:
X = idf[['n_down_7','std_4', 'val_4', 'val_8', 'Hladist_1']]
y = idf.flretC_6
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   random_state = 0)


linreg = LinearRegression().fit(X_train, y_train)

print('Crime dataset')
print('linear model intercept: {}'
     .format(linreg.intercept_))
print('linear model coeff:\n{}'
     .format(linreg.coef_))
print('R-squared score (training): {:.3f}'
     .format(linreg.score(X_train, y_train)))
print('R-squared score (test): {:.3f}'
     .format(linreg.score(X_test, y_test)))

Crime dataset
linear model intercept: 0.010972355604745304
linear model coeff:
[-4.94281261e-04  1.06809155e-07 -1.83281766e+01  1.04354211e+01
  0.00000000e+00]
R-squared score (training): 0.002
R-squared score (test): 0.001


In [166]:

X = idf[['n_down_7','n_down_7','val_8', 'Hladist_1','wiLdist']]
y = idf.candle_dir
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   random_state = 0)


scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


linridge = Ridge(alpha=20.0).fit(X_train_scaled, y_train)

print('ridge regression linear model intercept: {}'
     .format(linridge.intercept_))
print('ridge regression linear model coeff:\n{}'
     .format(linridge.coef_))
print('R-squared score (training): {:.3f}'
     .format(linridge.score(X_train_scaled, y_train)))
print('R-squared score (test): {:.3f}'
     .format(linridge.score(X_test_scaled, y_test)))
print('Number of non-zero features: {}'
     .format(np.sum(linridge.coef_ != 0)))

ridge regression linear model intercept: -1.8736919714572378
ridge regression linear model coeff:
[-0.32709302 -0.32709302  3.84799281  0.          0.67069697]
R-squared score (training): 0.138
R-squared score (test): 0.135
Number of non-zero features: 4


In [167]:
print('Ridge regression: effect of alpha regularization parameter\n')
for this_alpha in np.logspace(-4,3,8):
    linridge = Ridge(alpha = this_alpha).fit(X_train_scaled, y_train)
    r2_train = linridge.score(X_train_scaled, y_train)
    r2_test = linridge.score(X_test_scaled, y_test)
    num_coeff_bigger = np.sum(abs(linridge.coef_) > 1.0)
    print('Alpha = {:.2f}\nnum abs(coeff) > 1.0: {}, \
r-squared training: {:.2f}, r-squared test: {:.2f}\n'
         .format(this_alpha, num_coeff_bigger, r2_train, r2_test))

Ridge regression: effect of alpha regularization parameter

Alpha = 0.00
num abs(coeff) > 1.0: 2, r-squared training: 0.16, r-squared test: 0.15

Alpha = 0.00
num abs(coeff) > 1.0: 2, r-squared training: 0.16, r-squared test: 0.15

Alpha = 0.01
num abs(coeff) > 1.0: 2, r-squared training: 0.16, r-squared test: 0.15

Alpha = 0.10
num abs(coeff) > 1.0: 2, r-squared training: 0.16, r-squared test: 0.15

Alpha = 1.00
num abs(coeff) > 1.0: 2, r-squared training: 0.16, r-squared test: 0.15

Alpha = 10.00
num abs(coeff) > 1.0: 1, r-squared training: 0.15, r-squared test: 0.15

Alpha = 100.00
num abs(coeff) > 1.0: 1, r-squared training: 0.08, r-squared test: 0.08

Alpha = 1000.00
num abs(coeff) > 1.0: 0, r-squared training: 0.02, r-squared test: 0.02



In [168]:

linlasso = Lasso(alpha=0.1, max_iter = 10000).fit(X_train_scaled, y_train)

# print('Crime dataset')
print('lasso regression linear model intercept: {}'
     .format(linlasso.intercept_))
print('lasso regression linear model coeff:\n{}'
     .format(linlasso.coef_))
print('Non-zero features: {}'
     .format(np.sum(linlasso.coef_ != 0)))
print('R-squared score (training): {:.3f}'
     .format(linlasso.score(X_train_scaled, y_train)))
print('R-squared score (test): {:.3f}\n'
     .format(linlasso.score(X_test_scaled, y_test)))
print('Features with non-zero weight (sorted by absolute magnitude):')

for e in sorted (list(zip(list(X), linlasso.coef_)),
                key = lambda e: -abs(e[1])):
    if e[1] != 0:
        print('\t{}, {:.3f}'.format(e[0], e[1]))

lasso regression linear model intercept: 0.021947581504368208
lasso regression linear model coeff:
[-0. -0.  0.  0.  0.]
Non-zero features: 0
R-squared score (training): 0.000
R-squared score (test): -0.001

Features with non-zero weight (sorted by absolute magnitude):
